In [1]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from typing import Dict, Any, List, Tuple, Optional
import json

from lightgbm import LGBMRegressor  # Direkte Nutzung der sklearn-API

# --- 1. Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# --- Import necessary functions and models ---
from src.config import GlobalConfig, DEFAULT_CORR_SPEC, OUTPUTS
from src.io_timesplits import load_target, load_ifo_features
from src.features import (
    build_engineered_matrix,
    pw_corr,
)

print('PROJECT_ROOT =', PROJECT_ROOT)

# --- Output-Verzeichnis ---
BASE_OUTPUT_DIR = OUTPUTS / "feature_importance"
OUTPUT_DIR = BASE_OUTPUT_DIR / "output_incl_missing"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print('Output Directory =', OUTPUT_DIR)

# --- 2. Konfiguration ---
# Diese Datei nutzt ausschließlich LGBM mit Missing Values.
MAX_LAG = 6  # Lags 0..6 (0 = aktueller Monat, 1..6 Vergangenheit)

# --- Feste Hyperparameter für LGBM (Basis) ---
HP_LGBM = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'num_leaves': 31,
    'colsample_bytree': 0.1,   # wird als feature_fraction interpretiert
    'min_child_samples': 20,    # wird dynamisch "safe" gesetzt
    # keine Early Stopping-Params hier – wir trainieren ohne ES
}

# --- 3. Daten laden ---
y = load_target()
X_ifo = load_ifo_features()  # volle (ca. 2k+) Features

idx = y.index.intersection(X_ifo.index)
y, X_ifo = y.loc[idx], X_ifo.loc[idx]
T = len(y)
print('Daten geladen. Shapes:', X_ifo.shape, y.shape)


# --- 4a. Lag-Map: Lag 0 + bester Lag > 0 je Feature ---

def build_best_lag_map_global(
    X: pd.DataFrame,
    y: pd.Series,
    max_lag: int,
    corr_spec: Any,
) -> Dict[str, List[int]]:
    """
    Für jedes ifo-Feature:
      - berechnet die Korrelation von Lag ℓ (ℓ=1..max_lag) mit y_{t+1},
      - wählt den Lag ℓ*>0 mit maximaler |corr|,
      - gibt lag_map[col] = [0, ℓ*] (sortiert) zurück.

    Lag 0 bleibt IMMER enthalten, es kommt genau EIN bester Lag >0 dazu.
    """
    y_next = y.shift(-1)
    lag_map: Dict[str, List[int]] = {}

    for col in X.columns:
        s = X[col]
        best_lag = None
        best_score = -1.0

        for lag in range(1, max_lag + 1):
            x_lag = s.shift(lag)
            mask = (~x_lag.isna()) & (~y_next.isna())
            if mask.sum() < 3:
                continue

            score = abs(pw_corr(x_lag[mask].to_numpy(), y_next[mask].to_numpy(), corr_spec))
            if score > best_score:
                best_score = score
                best_lag = lag

        # Fallback: falls keine valide Korrelation berechnet werden konnte
        if best_lag is None:
            best_lag = 1 if max_lag >= 1 else 0

        lags_for_col = sorted({0, int(best_lag)})
        lag_map[col] = lags_for_col

    return lag_map


# --- 4b. Modifizierte Feature-Pipeline (Lag 0 + bester Lag >0, kein Top-K) ---

def run_feature_pipeline_for_importance(
    X: pd.DataFrame,
    y: pd.Series,
    t_origin: int,
    lag_map: Dict[str, List[int]],
) -> Tuple[pd.DataFrame, pd.Series, Dict[str, List[int]]]:
    """
    Führt FE durch (Lag 0 + bester Lag >0), KEIN RM3, KEIN Top-K.
    Ziel ist y(t+1). NaN/Inf-Behandlung: Inf -> NaN, NaNs bleiben erhalten (LGBM kann NaN).
    """
    I_t = t_origin + 1

    if lag_map is None or len(lag_map) == 0:
        return pd.DataFrame(), pd.Series(dtype=float), {}

    # Gelaggte Matrix bauen
    X_eng = build_engineered_matrix(X, lag_map)
    if X_eng.empty:
        return pd.DataFrame(), pd.Series(dtype=float), lag_map

    # maximal benötigter Lag aus Spaltennamen ("feature__lagℓ")
    def _lag_of(col):
        try:
            return int(str(col).split('__lag')[-1])
        except Exception:
            return 0

    head_needed = max([_lag_of(c) for c in X_eng.columns] + [0])

    # Basis-Zeitindizes wie in select_lags_per_feature: 1 .. I_t-1
    taus = np.arange(1, int(I_t) - 1, dtype=int)
    if taus.size == 0:
        return pd.DataFrame(), pd.Series(dtype=float), lag_map

    # nur Zeilen, wo y_{t+1} existiert
    y_next_base = y.shift(-1).iloc[taus].to_numpy(dtype=float)
    valid_y_mask = ~np.isnan(y_next_base)
    taus = taus[valid_y_mask]
    if taus.size == 0:
        return pd.DataFrame(), pd.Series(dtype=float), lag_map

    # Nur Zeilen, die genug Historie für den max Lag haben
    taus_model_mask = (taus - head_needed >= 0)
    if np.sum(taus_model_mask) == 0:
        taus_model = taus[-1:].copy() if taus.size > 0 else np.array([], dtype=int)
    else:
        taus_model = taus[taus_model_mask]

    if len(taus_model) == 0:
        return pd.DataFrame(), pd.Series(dtype=float), lag_map

    X_train_final = X_eng.iloc[taus_model, :].copy()
    y_train_final = y.shift(-1).iloc[taus_model].copy()

    # Inf -> NaN, NaNs bleiben (LightGBM kann NaNs nativ handhaben)
    X_train_final = X_train_final.replace([np.inf, -np.inf], np.nan)

    return X_train_final, y_train_final, lag_map


# --- Job 1: Vollständige gelaggte Feature-Matrix speichern (mit NaNs) ---
print("\n" + "="*20)
print("--- STARTING JOB 1: Build and save full lagged feature matrix ---")
print("="*20)

t_ref_full = T - 2  # Referenzmonat, aber Lag-Wahl ist global
lag_map_full = build_best_lag_map_global(
    X_ifo,
    y,
    max_lag=MAX_LAG,
    corr_spec=DEFAULT_CORR_SPEC,
)
print(f"Lag map (Lag 0 + bester Lag>0, max Lag={MAX_LAG}) erstellt (global).")

X_eng_full = build_engineered_matrix(X_ifo, lag_map_full)
X_eng_full = X_eng_full.replace([np.inf, -np.inf], np.nan)  # NaNs bleiben

lagged_features_path = OUTPUT_DIR / "X_eng_full_lagged.parquet"
X_eng_full.to_parquet(lagged_features_path)
print(f"Full lagged feature matrix (with NaNs) saved to: {lagged_features_path}")
print(f"Shape of saved matrix: {X_eng_full.shape}")
# --- ENDE JOB 1 ---


def perform_rolling_importance(start_origin: int, end_origin: int, run_name: str) -> Tuple[pd.DataFrame, Dict[str, pd.DataFrame]]:
    """
    Speicher- und laufzeitfreundliche Version:
    - Nutzt festen Spaltenraum (aus X_eng_full.columns)
    - schreibt die Roh-Importances gestreamt in eine CSV (Chunkweise)
    - aggregiert online (laufender Mittelwert) -> kein Großobjekt im RAM
    """
    import gc
    all_raw_dfs: Dict[str, pd.DataFrame] = {}

    # Fester Spaltenraum aus der vollständigen gelaggten Matrix
    feature_cols = list(X_eng_full.columns)
    n_cols = len(feature_cols)
    col_index = {c: i for i, c in enumerate(feature_cols)}

    # Output-Datei für die Rohwerte (überschreiben)
    raw_path = OUTPUT_DIR / f"raw_importance_LGBM_{run_name}.csv"
    if raw_path.exists():
        raw_path.unlink()

    # Online-Aggregation (float64 für Stabilität)
    sum_vec = np.zeros(n_cols, dtype=np.float64)
    n_rows_agg = 0

    # Chunk-Puffer für Streaming
    chunk_rows: List[np.ndarray] = []
    chunk_index: List[int] = []
    CHUNK_SIZE = 25  # alle 25 Iterationen flushen

    origins = range(start_origin, end_origin + 1)
    for t in tqdm(origins, desc=f"Rolling Importance ({start_origin}-{end_origin})"):
        X_tr, y_tr, _ = run_feature_pipeline_for_importance(X_ifo, y, t, lag_map=lag_map_full)
        if X_tr.empty or y_tr.empty or len(y_tr) < 20:
            continue

        # WICHTIG: auf gemeinsamen Spaltenraum bringen (fehlende Spalten = 0)
        # (NaNs in X_tr bleiben erlaubt)
        X_tr = X_tr.reindex(columns=feature_cols, fill_value=0.0)

        # LGBM mit sparsamen Settings
        n_train = len(y_tr)
        safe_min_child = max(1, min(HP_LGBM['min_child_samples'], (n_train - 1) // 2))
        lgbm = LGBMRegressor(
            n_estimators=min(80, HP_LGBM['n_estimators']),   # sparsamer
            learning_rate=HP_LGBM['learning_rate'],
            num_leaves=min(31, 2*safe_min_child+1),
            max_depth=-1,
            feature_fraction=min(0.6, HP_LGBM.get('colsample_bytree', 0.6)),
            bagging_fraction=0.7,
            bagging_freq=1,
            max_bin=63,
            min_child_samples=safe_min_child,
            reg_lambda=0.0,
            random_state=42,
            n_jobs=1
        )
        lgbm.fit(X_tr, y_tr)

        booster = lgbm.booster_
        if booster.num_trees() == 0:
            # nichts gelernt -> 0-Zeile
            row = np.zeros(n_cols, dtype=np.float32)
        else:
            gain = booster.feature_importance(importance_type="gain")
            names = booster.feature_name()
            # Map {name -> gain} -> fester Vektor (float32)
            tmp = dict(zip(names, gain))
            row = np.zeros(n_cols, dtype=np.float32)
            # nur über Features iterieren, die im Booster vorkommen
            for name, g in tmp.items():
                idx = col_index.get(name)
                if idx is not None:
                    row[idx] = np.float32(g)

        # Online-Aggregation
        sum_vec += row.astype(np.float64, copy=False)
        n_rows_agg += 1

        # In Chunk puffern
        chunk_rows.append(row)
        chunk_index.append(t)

        # Periodisch auf Platte schreiben
        if len(chunk_rows) >= CHUNK_SIZE:
            df_chunk = pd.DataFrame(np.vstack(chunk_rows), index=chunk_index, columns=feature_cols)
            df_chunk.to_csv(raw_path, mode='a', header=not raw_path.exists())
            chunk_rows.clear()
            chunk_index.clear()

        # Speicher freigeben
        del lgbm, booster, row, X_tr, y_tr, gain, names
        gc.collect()

    # Rest-Chunk flushen
    if chunk_rows:
        df_chunk = pd.DataFrame(np.vstack(chunk_rows), index=chunk_index, columns=feature_cols)
        df_chunk.to_csv(raw_path, mode='a', header=not raw_path.exists())
        chunk_rows.clear()
        chunk_index.clear()

    print(f"Rolling Importance Calculation ({start_origin}-{end_origin}) Done.")
    print(f"Raw LGBM importances written to: {raw_path}")

    # Aggregiertes Ergebnis als DataFrame
    if n_rows_agg == 0:
        print("Warning: No rows aggregated.")
        df_agg = pd.DataFrame(index=feature_cols, data={'LGBM': 0.0})
    else:
        mean_vec = (sum_vec / n_rows_agg)
        # pro Modell normalisieren (Summe = 1, falls > 0)
        ssum = mean_vec.sum()
        if ssum > 0:
            mean_vec = mean_vec / ssum
        df_agg = pd.DataFrame({'LGBM': mean_vec}, index=feature_cols)

    all_raw_dfs = {'LGBM': None}  # wir haben direkt auf Platte gestreamt
    return df_agg, all_raw_dfs


# ======================================================================
# --- RUN 1: Ex-Ante + Ex-Post (t=120 bis T-2) ---
# ======================================================================
print("\n" + "="*20)
print("--- STARTING FULL IMPORTANCE RUN (t=120 to T-2) ---")
print("="*20)

START_ORIGIN_FULL = 120
END_ORIGIN_FULL = T - 2

try:
    df_agg_norm_full, raw_dfs_full = perform_rolling_importance(START_ORIGIN_FULL, END_ORIGIN_FULL, "full_run")
except Exception as e:
    print("perform_rolling_importance crashed with:", repr(e))
    raise

# --- Output 1: Dynamische LGBM-Top-Listen nach kumulativer Wichtigkeit ---
if not df_agg_norm_full.empty and 'LGBM' in df_agg_norm_full.columns:
    lgbm_scores_sorted = df_agg_norm_full['LGBM'].sort_values(ascending=False)
    cumulative_importance_pct = lgbm_scores_sorted.cumsum()

    thresholds_to_create = [0.50, 0.90, 0.99]
    print(f"\n--- Erstelle LGBM Feature-Listen für Schwellenwerte: {thresholds_to_create} ---")

    for thresh in thresholds_to_create:
        k_dynamic = np.searchsorted(cumulative_importance_pct.values, thresh, side='right')
        k_dynamic = max(1, int(k_dynamic))
        final_top_features = lgbm_scores_sorted.head(k_dynamic).index.tolist()

        print(f"Dynamisch ausgewählt: Top {k_dynamic} Features (für {thresh*100:.0f}% kum. Wichtigkeit)")
        # kurze Vorschau
        for i, feat in enumerate(final_top_features[:5]):
            print(f"{i+1}. {feat}")
        if len(final_top_features) > 5:
            print("...")

        thresh_str = str(thresh).replace(".", "_")
        output_path = OUTPUT_DIR / f"top_features_LGBM_full_thresh_{thresh_str}.json"
        with open(output_path, "w") as f:
            json.dump(final_top_features, f, indent=2)
        print(f"Saved LGBM list ({len(final_top_features)} Features) to: {output_path}")
else:
    print("Keine aggregierten LGBM-Importances vorhanden – überspringe Top-Listen.")


# ======================================================================
# --- Job 3: Rolling Mean Importance (60m) – nur LGBM ---
# ======================================================================
print("\n" + "="*50)
print("--- STARTING JOB 3: Build and save Rolling Mean Importance (60m) ---")
print("="*50)

ROLLING_WINDOW_SIZE = 60
run_name = "full_run"

raw_path = OUTPUT_DIR / f"raw_importance_LGBM_{run_name}.csv"
if not raw_path.exists():
    print("Keine rohen LGBM-Wichtigkeitsdateien gefunden. Überspringe Job 3.")
else:
    df_raw = pd.read_csv(raw_path, index_col=0)

    # Zeilenweise normalisieren (Summe jeder Zeile = 1, falls >0)
    row_sums = df_raw.sum(axis=1).replace(0.0, np.nan)
    df_norm = df_raw.div(row_sums, axis=0).fillna(0.0)

    # Rolling 60m
    df_rolling_mean_imp = df_norm.rolling(window=ROLLING_WINDOW_SIZE, min_periods=1).mean()

    # Ziel-Daten (t+1) als Index setzen – nur wenn Länge matcht
    target_dates = y.index[START_ORIGIN_FULL + 1 : END_ORIGIN_FULL + 2]

    if len(target_dates) == len(df_rolling_mean_imp):
        df_rolling_mean_imp.index = target_dates
        df_rolling_mean_imp = df_rolling_mean_imp.reindex(y.index)

        rolling_imp_path = OUTPUT_DIR / f"rolling_mean_importance_{ROLLING_WINDOW_SIZE}m.parquet"
        df_rolling_mean_imp.to_parquet(rolling_imp_path)

        print(f"\nRolling Mean Importance (Shape: {df_rolling_mean_imp.shape}) gespeichert in:")
        print(rolling_imp_path)
        print("\nBeispiel-Daten (erste gültige Zeile bei t=180):")
        print(df_rolling_mean_imp.loc[y.index[180]:].dropna().head())
    else:
        print(f"Hinweis: Länge der Zieldaten ({len(target_dates)}) != Länge der Rolling-Datei ({len(df_rolling_mean_imp)}).")
        print("Die normalisierte LGBM-Zeitreihe (CSV) ist vorhanden; Parquet-Ausgabe wird deshalb übersprungen.")

print("\nFeature Importance Analysis Complete.")



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Output Directory = /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/feature_importance/output_incl_missing
INFO in load_ifo_features: Renaming columns to ensure validity.
Daten geladen. Shapes: (407, 2160) (407,)

--- STARTING JOB 1: Build and save full lagged feature matrix ---
Lag map (Lag 0 + bester Lag>0, max Lag=6) erstellt (global).
Full lagged feature matrix (with NaNs) saved to: /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/feature_importance/output_incl_missing/X_eng_full_lagged.parquet
Shape of saved matrix: (407, 4320)

--- STARTING FULL IMPORTANCE RUN (t=120 to T-2) ---


Rolling Importance (120-405):   0%|          | 1/286 [00:00<01:52,  2.54it/s]

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 166655
[LightGBM] [Info] Number of data points 

Rolling Importance (120-405):   1%|          | 2/286 [00:00<01:46,  2.66it/s]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Rolling Importance (120-405): 100%|█████████▉| 285/286 [02:15<00:00,  1.55it/s]

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000389 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 272160
[LightGBM] [Info] Number of data points 

Rolling Importance (120-405): 100%|██████████| 286/286 [02:15<00:00,  2.10it/s]


Rolling Importance Calculation (120-405) Done.
Raw LGBM importances written to: /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/feature_importance/output_incl_missing/raw_importance_LGBM_full_run.csv

--- Erstelle LGBM Feature-Listen für Schwellenwerte: [0.5, 0.9, 0.99] ---
Dynamisch ausgewählt: Top 458 Features (für 50% kum. Wichtigkeit)
1. Erzeugung_und_erste_Bearbeitung_von_NE_Metallen.Nachfrage_gegen_Vormonat__lag0
2. Herstellung_von_elektrischem_Installationsmaterial.Gesch_ftslage_Erwartungen__lag0
3. Herstellung_von_Schleifk_rpern_und_Schleifmitteln.Gesch_ftslage_Erwartungen__lag0
4. Herstellung_von_Teilen_und_Zubeh_r_f_r_Kraftwagen.Produktionspl_ne__lag0
5. Veredelung_und_Bearbeitung_von_Flachglas.Nachfrage_gegen_Vormonat__lag2
...
Saved LGBM list (458 Features) to: /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/feature_importance/output_incl_missing/top_features_LGBM_full_thresh_0_5.json
Dynamisch ausgewählt: Top 2125 Features (für 90% kum. Wichtigkeit)
1. 